In [2]:
import numpy as np
import timeit

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
## ALL MAIN ARGUMENTS FOR THE SCRIPT ##

dat_folder = '../data/'
img_folder = '../data/images/'
num_workers = 4

filter_subset = False # True if we want to filter to just train _1

## THIS VERSION OF SCRIPT HAS EQUAL NUMBER OF PAINTINGS PER ARTIST
num_train = 240
num_val = 30
num_test = 30
num_samples = num_train + num_val + num_test # threshold to include an artist
b_size = 60 # batch size for the data loaders


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image, ImageFilter
import random
import cv2
import os, glob

t = pd.read_csv(dat_folder + 'all_data_info.csv')

# filter down (if needed)
if (filter_subset):
    t = t[t['new_filename'].str.startswith('1')]
    t = t[t['in_train']]

t.head()
# print(t.shape)

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [23]:
temp = t['artist'].value_counts()
artists = temp[temp >= num_samples].index.tolist()

ta = t[t['artist'].isin(artists)]

art_style = ta[['artist', 'style']]
art_style = art_style.sort_values(['artist'])
art_style = art_style.drop_duplicates()
print(art_style)

                           artist                       style
10527            Albert Bierstadt                 Romanticism
31932            Albert Bierstadt                    Luminism
51363            Albert Bierstadt                     Realism
2517               Albrecht Durer        Northern Renaissance
5611               Albrecht Durer                         NaN
5187                Alfred Sisley               Impressionism
5219                Alfred Sisley                     Realism
36662               Alfred Sisley          Post-Impressionism
5933            Amedeo Modigliani               Expressionism
44626             Boris Kustodiev        Art Nouveau (Modern)
54781             Boris Kustodiev           Socialist Realism
51093             Boris Kustodiev                     Realism
55060             Boris Kustodiev     Naïve Art (Primitivism)
53335             Boris Kustodiev               Impressionism
7231              Boris Kustodiev                         NaN
47867   